In [0]:
# Transfer learning using VGG

In [0]:
'''
importing necessary libraries
'''
import math
from shutil import copyfile
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt # for plotting
import os
import sys

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# for importing VGG pretrained model
import torchvision.models
vgg = torchvision.models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


##Regular Data Loader

In [0]:
# get dataloaders using the 1000-image dataset
def get_data_loader(batch_size=32):

    np.random.seed(1000) # set the seed for reproducible shuffling
    num_workers = 1

    # define the training, validation, and testing directories to the smaller dataset
    train_path = '/content/drive/My Drive/Colab Notebooks/APS360/APS360 Project/food-11/training'
    valid_path = '/content/drive/My Drive/Colab Notebooks/APS360/APS360 Project/food-11/validation'
    test_path = '/content/drive/My Drive/Colab Notebooks/APS360/APS360 Project/food-11/evaluation'

    # convert all jpgs to tensors
    data_transform = transforms.Compose([transforms.Resize((224,224)), 
                                    transforms.ToTensor()])

    # load training, validation, and testing data
    train_data = torchvision.datasets.ImageFolder(root = train_path, 
                                            transform=data_transform)

    val_data = torchvision.datasets.ImageFolder(root = valid_path, 
                                            transform=data_transform)

    test_data = torchvision.datasets.ImageFolder(root = test_path, 
                                            transform=data_transform)
    
    # get dataset loaders
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)

    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)

    return train_loader, val_loader, test_loader

##Feature Loader
- Which I don't think I ever call but whatever

In [0]:
# get the feature loaders
def get_feature_loaders(batch_size):
    trainLoader, valLoader, testLoader = get_data_loader(batch_size)

    trainFeatures, valFeatures, testFeatures = []
    trainLabels, valLabels, testLabels = []

    for i, data in enumerate(trainLoader, 1):
        # Get the inputs
        inputs, labels = data
        trainFeatures.append(inputs)
        trainLabels.append(labels)

    for i, data in enumerate(valLoader, 1):
        # Get the inputs
        inputs, labels = data
        valFeatures.append(inputs)
        valLabels.append(labels)

    for i, data in enumerate(testLoader, 1):
        # Get the inputs
        inputs, labels = data
        testFeatures.append(inputs)
        testLabels.append(labels)

    return trainFeatures, valFeatures, testFeatures, trainLabels, valLabels, testLabels

##Save Features to a Folder
- Prevents us from recomputing the features every time the thing is run
- ALREADY RAN IT ONCE DO NOT RUN THIS AGAIN UNLESS YOU'RE PREPARED TO WAIT 3 HOURS

In [0]:
# Save Features to Folder (assumes code from 1. has been evaluated)

# location on Google Drive
master_path = os.getcwd() + "/drive/My Drive/Colab Notebooks/APS360/APS360 Project/VGG"
#print(os.getcwd())
#os.chdir('/content/drive/My Drive/Colab Notebooks/APS360/APS360 Project/VGG')
#!ls

train_loader, val_loader, test_loader = get_data_loader(1)

# the food categories in the dataset
classes = ['Bread', 'Dairy', 'VegetableFruit', 'Dessert', 'Egg', 'FriedFood', 'Meat', 'NoodlesPasta', 'Rice', 'Seafood', 'Soup']

# save features to folder as tensors
i = 0
for img, label in train_loader:
  features = vgg.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())
  
  #print(str(classes[label]))

  folder_name = master_path + '/train/' +  str(classes[label])
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  torch.save(features_tensor.squeeze(0), folder_name + '/' + str(i) + '.tensor')
  i += 1

j = 0
for img, label in val_loader:
  features = vgg.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())

  folder_name = master_path + '/val/' +  str(classes[label])
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  torch.save(features_tensor.squeeze(0), folder_name + '/' + str(j) + '.tensor')
  j += 1

k = 0
for img, label in test_loader:
  features = vgg.features(img)
  features_tensor = torch.from_numpy(features.detach().numpy())

  folder_name = master_path + '/test/' +  str(classes[label])
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
  torch.save(features_tensor.squeeze(0), folder_name + '/' + str(k) + '.tensor')
  k += 1

##VGG Data Loader

In [0]:
def vgg_data_loader(batch_size):
    np.random.seed(1000) # set the seed for reproducible shuffling

    master_path = '/content/drive/My Drive/Colab Notebooks/APS360/APS360 Project/VGG'
    vgg_train_path = master_path + '/train'
    vgg_val_path = master_path + '/val'
    vgg_test_path = master_path + '/test'

    vgg_train_dataset = torchvision.datasets.DatasetFolder(vgg_train_path, loader=torch.load, extensions=('.tensor'))
    vgg_val_dataset = torchvision.datasets.DatasetFolder(vgg_val_path, loader=torch.load, extensions=('.tensor'))
    vgg_test_dataset = torchvision.datasets.DatasetFolder(vgg_test_path, loader=torch.load, extensions=('.tensor'))

    # Prepare Dataloader
    num_workers = 1
    vgg_train_loader = torch.utils.data.DataLoader(vgg_train_dataset, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
    vgg_val_loader = torch.utils.data.DataLoader(vgg_val_dataset, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
    vgg_test_loader = torch.utils.data.DataLoader(vgg_test_dataset, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
    return vgg_train_loader, vgg_val_loader, vgg_test_loader

##Verification Step
- To determine the size of the input images

In [22]:
# Verification Step - obtain one batch of features

sample_stuff, more_sample_stuff, other_sample_stuff = vgg_data_loader(32)

dataiter = iter(sample_stuff)
features, labels = dataiter.next()
print("features dimensions:", features.shape)
print("labels dimensions:", labels.shape)
print(labels)

dataiter = iter(more_sample_stuff)
features, labels = dataiter.next()
print("features dimensions:", features.shape)
print("labels dimensions:", labels.shape)
print(labels)

dataiter = iter(other_sample_stuff)
features, labels = dataiter.next()
print("features dimensions:", features.shape)
print("labels dimensions:", labels.shape)
print(labels)

features dimensions: torch.Size([32, 512, 7, 7])
labels dimensions: torch.Size([32])
tensor([ 8, 10,  4, 10,  2, 10, 10,  2,  2,  3,  4,  9,  2,  1,  9,  1,  2, 10,
         4,  5,  2,  0, 10,  6,  2,  6,  9,  9,  9, 10,  4,  6])
features dimensions: torch.Size([32, 512, 7, 7])
labels dimensions: torch.Size([32])
tensor([10, 10,  4,  7,  4,  3,  6, 11,  6,  9,  4, 10,  0,  2,  0, 10,  2,  0,
         0, 10, 10,  8,  1,  2,  8, 11, 10,  9,  3,  4,  2,  0])
features dimensions: torch.Size([32, 512, 7, 7])
labels dimensions: torch.Size([32])
tensor([ 1,  5,  6,  3,  0,  2,  6,  6,  2, 10,  8,  9,  8,  8,  3,  9,  8,  6,
         5,  0,  4,  6,  9,  8,  6, 10, 10,  3,  1,  0,  7,  7])


##Neural Network Architecture
- Just an ANN for now, add CNN later maybe???

In [0]:
#Artifical Neural Network Architecture
# ----------------------------------CALCULATIONS----------------------------------
# there are 512 7x7 input images and 11 expected outputs
class FoodVGG(nn.Module):
    def __init__(self, hidden_size):
        super(FoodVGG, self).__init__()
        self.name = "FoodVGG"

        self.hidden_size = hidden_size

        self.fc1 = nn.Linear(512 * 7 * 7, hidden_size)
        #self.fc3 = nn.Linear(2048, 512)
        #self.fc4 = nn.Linear(512, 32)
        self.fc2 = nn.Linear(hidden_size, 11)

    def forward(self, x):
        x = x.view(-1, 512 * 7 * 7) #flatten feature data
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        x = self.fc2(x)
        return x

##Calculate Accuracy

In [0]:
def get_accuracy_vgg(model, loader, loss_function):
    correct = 0
    loss2 = 0
    num_evaluated = 0

    for num_batches, data in enumerate(loader, 1):
        imgs, labels = data
        #imgs = alexnet.features(imgs)

        if torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()

        # determine accuracy
        prediction = model(imgs)
        pred = prediction.max(1, keepdim=True)[1] #select index with maximum prediction score
        correct += pred.eq(labels.view_as(pred)).sum().item()

        # determine loss
        loss1 = loss_function(prediction, labels.long())
        loss2 += loss1.item()

        num_evaluated += len(labels) # this is how many labels you just evaluated

    # accuracy: total accuracy / number of items evaluated
    accuracy_rate = float(correct) / num_evaluated 
    # loss: total loss / batch size evaluated
    loss_rate = float(loss2) / num_batches
    
    return accuracy_rate, loss_rate

##Plot Graphs

In [0]:
def plot_graph(graph_title, x_label, y_label, num_epochs, training_data, val_data, testing_data = None):
    plt.figure()
    plt.title(graph_title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    
    plt.plot(range(1,num_epochs+1), training_data, label="Training")
    plt.plot(range(1,num_epochs+1), val_data, label="Validation")

    if testing_data != None:
        plt.plot(range(1,num_epochs+1), testing_data, label="Testing")
    plt.legend()
    plt.show()

##Get Model Path

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    return path

##Training Function

In [0]:
def train_vgg(model, batch_size=64, learning_rate = 0.01, num_epochs=30):
    np.random.seed(1000) # set the seed for reproducible shuffling

    # load the correct data
    train_loader, val_loader, test_loader = vgg_data_loader(batch_size)
    
    one_sample = iter(train_loader)

    criterion = nn.CrossEntropyLoss()
    print("Loss function used: CrossEntropyLoss")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print("Optimizer used: Adam")

    # training
    n = 0 # the number of iterations
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    ########################################################################
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    #count = 0
    start_time = time.time()
    #for epoch in range(num_epochs):  # loop over the dataset multiple times
    #for data in train_loader:
    #count += 1

    #if count == 1000:
        #count = 0
        #break

    # Get the inputs
    inputs, labels = one_sample.next()

    #############################################
    #To Enable GPU Usage
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()
    #############################################

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass, backward pass, and optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels.long())
    loss.backward()
    optimizer.step()

    train_err[0], train_loss[0] = get_accuracy_vgg(model, train_loader, criterion) # accuracy function provided
    #val_err[0], val_loss[0] = get_accuracy_vgg(model, val_loader, criterion)

    # Save the current model (checkpoint) to a file
    model_path = get_model_name(model.name, batch_size, learning_rate, 0)
    torch.save(model.state_dict(), model_path)

    print('\nFinished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

    print("\n-------------------------------------------------------------------------")
    print("Training accuracy after {} epochs: {}".format(num_epochs, 1-train_err[-1]))
    print("Training loss after {} epochs: {}".format(num_epochs, train_loss[-1]))
    print("\n-------------------------------------------------------------------------")
    #print("Validation accuracy after {} epochs: {}".format(num_epochs, val_err[-1]))
    #print("Validation loss after {} epochs: {}".format(num_epochs, val_loss[-1]))

    # print("\n------------------------------GRAPHS------------------------------------")
    # print("\nAccuracy plot of FoodVGG NN")   
    # plot_graph("Accuracy", "Number of Epochs", "Accuracy", num_epochs, train_err, val_err)

    # print("\nLoss plot of FoodVGG using CrossEntropyLoss")
    # plot_graph("Loss", "Number of Epochs", "Loss", num_epochs, train_loss, val_loss)

##Training and Hyperparameter Search

In [34]:
# batch size: 64
# learning rate: 0.0001
# number of epochs: 30
# number of layers: 1 fully-connected layer

model = FoodVGG(4096)

if torch.cuda.is_available():
    print("Using GPU...")
    model = model.cuda()
else:
    print("Using CPU...")

train_vgg(model, batch_size = 64, learning_rate = 0.001, num_epochs=5)

Using CPU...
Loss function used: CrossEntropyLoss
Optimizer used: Adam

Finished Training
Total time elapsed: 61.66 seconds

-------------------------------------------------------------------------
Training accuracy after 5 epochs: 0.0
Training loss after 5 epochs: 0.0

-------------------------------------------------------------------------


##Running on Testing Dataset

In [20]:
_, _, test_loader = vgg_data_loader(batch_size = 64)

model = FoodVGG(4096)

if torch.cuda.is_available():
    print("Using GPU...")
    model = model.cuda()
else:
    print("Using CPU...")

model_path = get_model_name(model.name, batch_size=64, learning_rate=0.001, epoch=4)
state = torch.load(model_path)
model.load_state_dict(state)

accuracy, loss = get_accuracy_vgg(model, test_loader, nn.CrossEntropyLoss())

print("The testing accuracy is:", accuracy)
print("The testing loss is:", loss)

Using CPU...


FileNotFoundError: ignored